# 🌟 Introduction à l’IA générative et au RAG

Ce projet a pour objectif de découvrir les bases de l'**IA générative** et d'expérimenter un premier **pipeline de RAG** (*Retrieval-Augmented Generation*).

## 🎯 Objectifs
- Comprendre la génération de texte avec un modèle open-source.
- Découvrir les **embeddings** et la **recherche vectorielle**.
- Créer un index FAISS pour retrouver des passages similaires.
- (Optionnel) Combiner recherche + génération pour construire un mini système RAG.


In [1]:
!pip install transformers sentence-transformers faiss-cpu torch -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 27.6 MB/s eta 0:00:00


## 🧩 Partie 1 — Génération de texte avec un modèle open-source

Dans cette partie, on découvre comment générer du texte à l’aide du pipeline `text-generation` de la librairie **Transformers**.
Nous utilisons ici le modèle **GPT-2**, l’un des premiers grands modèles de langage open-source.

In [3]:
from transformers import pipeline

# Création du pipeline de génération
generator = pipeline("text-generation", model="gpt2")

# Exemple de génération de texte
prompt = "Artificial intelligence will change the world because"
result = generator(prompt, max_length=60, num_return_sequences=1)

print(result[0]["generated_text"])

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=60) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Artificial intelligence will change the world because we're going to do it.

When I spoke to my friend on his podcast last year, he said he realized that AI would be the next big technological innovation. AI will change the world because we're going to do it. And that's just part of the story.

I think people are going to have to understand that when you're talking about the next big tech, you're talking about the next big thing. But if the next big thing is something that people care about, it will be something that people care about.

This is a major breakthrough, and it's important for humans to know that. And it's also important for AI.

The other issue that seems to come up is the question of which side of the fence you're on. I don't think there's any clear-cut answer that separates the two.

There's no clear-cut answer. The point is that there's a lot of work to do. There's very little chance that we will get there. And so we're going to be making progress.

So I think it's impo

## 📖 Partie 2 — Embeddings et recherche vectorielle avec FAISS

Les **embeddings** sont des représentations numériques du texte : chaque phrase est transformée en un vecteur dans un espace de grande dimension.  
Les phrases ayant un sens proche ont des vecteurs proches entre eux.

Nous allons :
1. Créer un petit corpus de textes.
2. Générer leurs embeddings avec un modèle SentenceTransformer.
3. Créer un index **FAISS** pour stocker et rechercher efficacement ces vecteurs.

In [4]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Exemple de petit corpus
corpus = [
    "Artificial intelligence is a discipline within computer science.",
    "Large language models can generate text.",
    "FAISS enables efficient vector search.",
    "Embeddings represent the meaning of sentences in the form of vectors.",
    "RAG combines research and generation to improve accuracy."
]

# Chargement du modèle d'embeddings
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Génération des vecteurs
embeddings = model.encode(corpus)

# Création d’un index FAISS
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

print("Index créé avec", index.ntotal, "documents.")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Index créé avec 5 documents.


## 🔍 Partie 3 — Recherche de passages similaires

Nous allons maintenant tester notre index FAISS pour retrouver les passages les plus proches d'une requête donnée.

In [5]:
# Exemple de requête
query = "How to combine search and text generation?"

# Calcul de l'embedding de la requête
query_embedding = model.encode([query])

# Recherche des k passages les plus proches
k = 2
distances, indices = index.search(np.array(query_embedding), k)

# Affichage des résultats
print("Requête :", query)
for i, idx in enumerate(indices[0]):
    print(f"\nRésultat {i+1}:")
    print(f"→ {corpus[idx]}")
    print(f"(distance = {distances[0][i]:.4f})")


Requête : How to combine search and text generation?

Résultat 1:
→ Large language models can generate text.
(distance = 0.9729)

Résultat 2:
→ RAG combines research and generation to improve accuracy.
(distance = 1.3474)


## 🤖 Partie 4 — Mini RAG : recherche + génération

Le **RAG (Retrieval-Augmented Generation)** combine la recherche d'information (retrieval)
et la génération de texte.  
L'idée est d'enrichir le prompt du modèle génératif avec les passages les plus pertinents
trouvés via FAISS.

Nous allons construire un mini RAG simplifié :
1. Trouver les passages les plus proches d'une question.
2. Les injecter dans le prompt du générateur de texte.

In [6]:
# Combiner les passages similaires en un seul contexte
context = " ".join([corpus[idx] for idx in indices[0]])

# Créer un prompt enrichi
prompt = f"Contexte : {context}\n\nQuestion : {query}\n\nRéponse :"

# Génération avec le modèle
response = generator(prompt, max_length=120, num_return_sequences=1)[0]["generated_text"]

print(response)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=120) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Contexte : Large language models can generate text. RAG combines research and generation to improve accuracy.

Question : How to combine search and text generation?

Réponse : We can use a search engine to generate a simple text. We can also use the word "search" to generate search results.

Question : What about the language model that generates these results?

Réponse : We can use the term "language model" to describe a real-world computer program. The process of transforming text into text is not limited to the computer. The program is also known as a machine learning algorithm.

Question : What about the model for text generation?

Réponse : We will analyze the text, but because we will not use any machine learning algorithms, we will use Google's machine learning program.

Question : Is there any way to model the text in a way that can predict its shape, size, and orientation?

Réponse : We will use a similar approach to model text. In the following text, we will use a different m

## ✅ Conclusion

Dans ce projet, nous avons :
- Généré du texte à l’aide d’un modèle open-source (GPT-2).
- Découvert les embeddings et la recherche vectorielle avec FAISS.
- Créé un index FAISS et recherché des passages similaires.
- (Optionnel) combiné la recherche et la génération dans un mini pipeline RAG.

🧠 Ce notebook illustre les bases de la **RAG pipeline**, une approche clé pour
rendre les modèles de langage plus précis et informés par des données externes.
